In [ ]:
from keras.models import Model, load_model
from keras.layers import Input
from one_hot import *
import numpy as np

In [2]:
MODEL_PATH = "sananmuunnin.h5"

In [3]:
one_hot_encoder = OneHot()
model = load_model(MODEL_PATH)

encoder_inputs = model.input[0]
encoder_outputs, state_h_enc, state_c_enc = model.layers[4].output
encoder_states = [state_h_enc, state_c_enc]
encoder_model = Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]
hidden_size = encoder_outputs.shape[-1]
decoder_state_input_h = Input(shape=(hidden_size,))
decoder_state_input_c = Input(shape=(hidden_size,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[5]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[6]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states
)

In [4]:
def predict(words):
    one_hot_encoded = one_hot_encoder.encode(words)[np.newaxis]
    state = encoder_model.predict(one_hot_encoded)
    
    decoder_output = np.zeros((1, 1, N_CHARACTERS))
    decoder_output[0, 0, START_TOKEN_INDEX] = 1
    output = []
    
    for i in range(len(words)):
        decoder_output, state_h, state_c = decoder_model.predict([decoder_output] + state)
        decoded_token_index = decoder_output[0, 0, :].argmax()

        if decoded_token_index == END_TOKEN_INDEX:
            break

        state = [state_h, state_c]
        decoder_output = np.zeros((1, 1, N_CHARACTERS))
        decoder_output[0, 0, decoded_token_index] = 1
        output.append(decoder_output[0, 0, :])

    return one_hot_encoder.decode(np.array(output))

In [20]:
test_words = [
    'ruskaa pakkasella',
    'huono vintti',
    'hellan vanki',
]

for words in test_words:
    print(words, '->', predict(words))

ruskaa pakkasella -> paskaa rukkasella
huono vintti -> vieno huntti
hellan vanki -> vallan henki
